In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('google_sheets_credentials.json', scope)
client = gspread.authorize(creds)

sheet = client.open('Name of your spreadsheet').sheet1

data = sheet.get_all_values()

data = np.array(data[1:])
data = data.astype(np.float32)

In [ ]:
X = data[:, :-1]
y = data[:, -1]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
input_size = X_train.shape[1]
hidden_size = 128
num_layers = 2
output_size = 1
num_epochs = 100
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = RNNModel(input_size, hidden_size, num_layers, output_size).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    inputs = torch.from_numpy(X_train).to(device)
    targets = torch.from_numpy(y_train).view(-1, 1).to(device)

    outputs = model(inputs)
    loss = criterion(outputs, targets)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
with torch.no_grad():
    inputs = torch.from_numpy(X_test).to(device)
    targets = torch.from_numpy(y_test).view(-1, 1).to(device)
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    print(f'MSE Loss on Test Data: {loss.item()}')

In [ ]:
model_keras = Sequential()
model_keras.add(LSTM(hidden_size, input_shape=(X_train.shape[1], 1)))
model_keras.add(Dense(1))
model_keras.compile(optimizer='adam', loss='mse')

In [ ]:
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
model_keras.fit(X_train_reshaped, y_train, epochs=num_epochs, batch_size=32, verbose=1)

In [ ]:
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
mse = model_keras.evaluate(X_test_reshaped, y_test)
print(f'MSE Loss on Test Data (Keras): {mse}')